In [2]:
!pip install torch
!pip3 install torchvision
!pip3 install deeplake
!pip install --upgrade urllib3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 554.4/554.4 kB 7.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 63.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB

In [2]:
import torch as torch
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import zipfile
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader,Subset
import deeplake
import torch.nn.functional as F

## Model Training and Testing

### Load Dataset

In [ ]:
class LeafDataset(Dataset):
  def __init__(self):#, file_path):
    super().__init__()
    self.ds = deeplake.load('hub://activeloop/plantvillage-with-augmentation')
    self.transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((350,350)),
        transforms.Normalize(0.5,0.5)

    ])
    total_data = 58775
    split_size = int(0.8*total_data)
    self.batch_size = 500 
    train_dataset = self.ds[:split_size]
    test_dataset = self.ds[split_size: total_data]
    self.train_data = test_dataset.pytorch(num_workers = 0, shuffle = True, transform = {'images': self.transform, 'labels': None}, batch_size = self.batch_size, decode_method = {'images': 'pil'})
    self.test_data = test_dataset.pytorch(num_workers = 0, shuffle = True, transform = {'images': self.transform, 'labels': None}, batch_size = self.batch_size, decode_method = {'images': 'pil'})
   
  def __len__(self):
    return len(self.ds)

  def __getitem__(self,index):
    return self.ds[index]


### TRAIN MODEL

In [ ]:

class LeafModel(torch.nn.Module):
  def __init__(self):
    super(LeafModel,self).__init__()
    # Define the layers
    self.conv1 = nn.Conv2d(3,10,kernel_size = 7)
    self.conv2 = nn.Conv2d(10,24,kernel_size = 7)
    self.conv3 = nn.Conv2d(24,64,kernel_size = 6)
    #self.conv4 = nn.Conv2D(40,64,kernel_size = 6)
    self.conv4 = nn.Conv2d(64,180,kernel_size = 6)
    self.fc1    = nn.Linear(35280,100)
    self.fc2    = nn.Linear(100,64)
    self.fc3    = nn.Linear(64,39)
    # Classes = 39

  def forward(self,x):
    # print(x)
    # print(x.shape)
    x = F.relu(self.conv1(x))
    x = F.max_pool2d(x, kernel_size = 5, stride = 2)
    x = F.relu(self.conv2(x))
    x = F.avg_pool2d(x, kernel_size = 5, stride = 2)
    x = F.relu(self.conv3(x))
    x = F.avg_pool2d(x, kernel_size = 5, stride = 2)
    x = F.relu(self.conv4(x))
    x = F.avg_pool2d(x, kernel_size = 5, stride = 2)
    # print(x.shape)
    x = x.view(x.size(0), -1)
    # print(x.shape)
    x = F.tanh(self.fc1(x))
    # print(x.shape)
    x = F.tanh(self.fc2(x))
    # print(x.shape)
    x = F.softmax(self.fc3(x, ),dim = -1)
    return x



# MAIN PROGRAM

dataset = LeafDataset()
model = LeafModel()

# criterion = nn.MSELoss()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = 0.001)

#------------------------------------------------------------------------------
#     TRAINING
#------------------------------------------------------------------------------
batch_size = dataset.batch_size
num_epochs = 6
total_train = int(58775*0.8)   # Total data points in the training dataset
all_losses = list()
all_accuracies = list()


# for images,labels in dataset.train_data:
#   model.train()
for epoch in range(num_epochs):
  correct = 0
  loss = 0.0
  model.train()
  for images,labels in dataset.train_data:
    optimizer.zero_grad()
    outputs = model(images)
    
    labels = torch.tensor([int(x[0]) for x in labels])#, dtype = torch.float32,requires_grad=True)
    # print(labels)
    
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    loss += loss.item()
    print(loss)

    # torch.max() returns two outputs, the maximum values and the corresponding index
    # We are intersted in the index, which is equivalent to the predicted digit
    _ ,predictions =  torch.max(outputs.data,1)     # Get maximum values in the output tensor, along the second dimension

    # Compare the predictions against labels, and count the correct ones
    correct += sum([predictions[i] == labels[i] for i in range(0,images.shape[0])])
    
    # print(correct)

  print('\n Loss for epoch ',epoch,': ',loss.item()/batch_size)
  print('\n Accuracy for epoch ',epoch,': ',(correct/total_train).numpy())
  all_losses.append(loss.item()/batch_size)
  all_accuracies.append(correct/total_train)

# PLOT THE LOSSES
plt.plot(list(range(num_epochs)),all_losses)
#for epoch,loss in zip(list(range(num_epochs)),all_losses):
#    plt.text(epoch, loss+500, str(int(loss)))
plt.xlabel("Epoch")
plt.ylabel("Losses")
plt.title("Plot of training loss after each epoch")
plt.show()

# PLOT THE ACCURACY
plt.plot(list(range(num_epochs)),all_accuracies,color= "orange")
#for epoch,accuracy in zip(list(range(num_epochs)),all_accuracies):
#    plt.text(epoch, accuracy+500, str(int(accuracy)))
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("Plot of training accuracy after each epoch")
plt.show()


\

Opening dataset in read-only mode as you don't have write permissions.


/

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/plantvillage-with-augmentation



/

hub://activeloop/plantvillage-with-augmentation loaded successfully.



/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
